# YOLO12 Model Analysis (n,s,m)

In [ ]:
!nvidia-smi 
!pip install ultralytics -q
!pip install numpy==1.26.0
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import yaml
import os
import cv2
from PIL import Image
import numpy as np
from IPython.display import display, HTML, clear_output

In [ ]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", "invalid value encountered in less", category=RuntimeWarning)
warnings.filterwarnings("ignore", "invalid value encountered in greater", category=RuntimeWarning)

## Validation YOLO12n

In [ ]:
from ultralytics import YOLO
modelN = YOLO("/kaggle/input/streetsignsensey12n/pytorch/streetsignsensey12n_361e_final/1/streetsignsense/yolo12n_run/weights/best.pt")
resultN = modelN.val(data="/kaggle/input/streetsignsensey12n/pytorch/streetsignsensey12n_361e_final/1/data.yaml", device='0,1')
print("\n\nmAP50-95:",resultN.box.map)  # map50-95
print("mAP-50:",resultN.box.map50)  # map50
print("mAP-75:",resultN.box.map75)  # map75
print("Average Precision: ", np.mean(resultN.box.p))  # precision
print("Average Recall: ",np.mean(resultN.box.r))  # recall
print("Average F1: ",np.mean(resultN.box.f1)) 

## Validation YOLO12s

In [ ]:
from ultralytics import YOLO
modelS = YOLO("/kaggle/input/streetsignsensey12s/pytorch/streetsignsensey12s_270e_final/1/streetsignsense/yolo12s_run/weights/best.pt")
resultS = modelS.val(data="/kaggle/input/streetsignsensey12s/pytorch/streetsignsensey12s_270e_final/1/data.yaml", device='0,1')
print("\n\nmAP50-95:",resultS.box.map)  # map50-95
print("mAP-50:",resultS.box.map50)  # map50
print("mAP-75:",resultS.box.map75)  # map75
print("Average Precision: ", np.mean(resultS.box.p))  # precision
print("Average Recall: ",np.mean(resultS.box.r))  # recall
print("Average F1: ",np.mean(resultS.box.f1)) #f1score

## Validation YOLO12m

In [ ]:
from ultralytics import YOLO
modelM = YOLO("/kaggle/input/streetsignsensey12m/pytorch/streetsignsensey12m_263e/1/streetsignsense/yolo12m_run/weights/best.pt")
resultM = modelM.val(data="/kaggle/input/streetsignsensey12m/pytorch/streetsignsensey12m_263e/1/data.yaml", device='0,1')
print("\n\nmAP50-95:",resultM.box.map)  # map50-95
print("mAP-50:",resultM.box.map50)  # map50
print("mAP-75:",resultM.box.map75)  # map75
print("Average Precision: ", np.mean(resultM.box.p))  # precision
print("Average Recall: ",np.mean(resultM.box.r))  # recall
print("Average F1: ",np.mean(resultM.box.f1)) #f1score

## Comparison of mAP between different models

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics = ['mAP-50-95', 'mAP-50', 'mAP-75']
models = ['Yolo12n', 'Yolo12s', 'Yolo12m']
values = np.array([
    [resultN.box.map, resultS.box.map, resultM.box.map],        # values for mAP-50-95
    [resultN.box.map50, resultS.box.map50, resultM.box.map50],  # values for mAP-50
    [resultN.box.map75, resultS.box.map75, resultM.box.map75]   # values for mAP-75
])
num_metrics = len(metrics)
num_models = len(models)

column_width = 0.2
gap = 0.1
position_bar = np.arange(num_metrics) * (num_models * column_width + gap)

colors = ['#F50C00', '#00FF66', '#007BFF']

fig, ax = plt.subplots(figsize=(10, 6))

for i in range(num_models):
    bars = ax.bar(position_bar + i * column_width, values[:, i], column_width, label=models[i], color=colors[i])
    ax.bar_label(bars, fmt='%.3f', padding=3, fontsize=9)

ax.set_xlabel('metrics')
ax.set_ylabel('Value')
ax.set_title('Comparison of mAP between different models')
ax.set_xticks(position_bar + (num_models - 1) * column_width / 2)
ax.set_xticklabels(metrics)
ax.legend(title='Models')
ax.set_ylim(0, np.max(values) * 1.1)

plt.tight_layout()
plt.show()
plt.close()

## Comparsion Training Loss and Validation Loss

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

path_csv_n = '/kaggle/input/streetsignsensey12n/pytorch/streetsignsensey12n_361e_final/1/streetsignsense/yolo12n_run/results.csv' # modello N
path_csv_s = '/kaggle/input/streetsignsensey12s/pytorch/streetsignsensey12s_270e_final/1/streetsignsense/yolo12s_run/results.csv' # modello S
path_csv_m = '/kaggle/input/streetsignsensey12m/pytorch/streetsignsensey12m_263e/1/streetsignsense/yolo12m_run/results.csv' # modello M

paths = [path_csv_n, path_csv_s, path_csv_m]
modelli = ['Yolo12n', 'Yolo12s', 'Yolo12m']

colors_val = ['#87CEFA', '#1E90FF', '#0000CD']
colors_train = ['#FFA07A', '#FF8C00', '#FF4500']

fig, ax = plt.subplots(figsize=(12, 7))

all_data_found = True
for i, path in enumerate(paths):
    try:
        data = pd.read_csv(path)

        data.columns = data.columns.str.strip()

        # YOLO salva più loss (box, cls, dfl).
        # Usiamo 'val/box_loss' e 'train/box_loss' come metriche di confronto.

        epoch = data['epoch']
        val_loss = data['val/box_loss']     # Puoi cambiarla con 'val/cls_loss'
        train_loss = data['train/box_loss'] # Puoi cambiarla con 'train/cls_loss'

        # Validation Loss (Blu)
        ax.plot(epoch, val_loss, label=f'Validation Loss {modelli[i]}', color=colors_val[i], linewidth=2, linestyle='-')
        # Training Loss (Arancione)
        ax.plot(epoch, train_loss, label=f'Training Loss {modelli[i]}', color=colors_train[i], linewidth=2, linestyle='-')

    except FileNotFoundError:
        print(f"--- ERRORE ---")
        print(f"File non trovato: '{path}'")
        print("-" * 30)
        all_data_found = False
    except KeyError as e:
        print(f"--- ERRORE ---")
        print(f"Errore nel file: {path}. Colonna non trovata: {e}")
        print("-" * 30)
        all_data_found = False

ax.set_xlabel('Epoche')
ax.set_ylabel('Box Loss (Train & Val)')
ax.set_title('Comparison of Training and Validation box Loss between YOLO12 Models')
ax.legend(title=' YOLO12 Models (n, m, s)')
ax.grid(True, linestyle='--', alpha=0.6)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if all_data_found:
    plt.tight_layout()
    plt.show()
    
fig, ax = plt.subplots(figsize=(12, 7))
all_data_found = True
for i, path in enumerate(paths):
    try:
        data = pd.read_csv(path)

        data.columns = data.columns.str.strip()

        # YOLO salva più loss (box, cls, dfl).
        # Usiamo 'val/box_loss' e 'train/box_loss' come metriche di confronto.

        epoch = data['epoch']
        val_loss = data['val/cls_loss']     # Puoi cambiarla con 'val/cls_loss'
        train_loss = data['train/cls_loss'] # Puoi cambiarla con 'train/cls_loss'

        # Validation Loss (Blu)
        ax.plot(epoch, val_loss, label=f'Validation Loss {modelli[i]}', color=colors_val[i], linewidth=2, linestyle='-')
        # Training Loss (Arancione)
        ax.plot(epoch, train_loss, label=f'Training Loss {modelli[i]}', color=colors_train[i], linewidth=2, linestyle='-')

    except FileNotFoundError:
        print(f"--- ERRORE ---")
        print(f"File non trovato: '{path}'")
        print("-" * 30)
        all_data_found = False
    except KeyError as e:
        print(f"--- ERRORE ---")
        print(f"Errore nel file: {path}. Colonna non trovata: {e}")
        print("-" * 30)
        all_data_found = False

ax.set_xlabel('Epoche')
ax.set_ylabel('Box Loss (Train & Val)')
ax.set_title('Comparison of Training and Validation cls Loss between YOLO12 Models')
ax.legend(title=' YOLO12 Models (n, m, s)')
ax.grid(True, linestyle='--', alpha=0.6)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.set_xlim(1, 10)
if all_data_found:
    plt.tight_layout()
    plt.show()
    
fig, ax = plt.subplots(figsize=(12, 7))
all_data_found = True
for i, path in enumerate(paths):
    try:
        data = pd.read_csv(path)

        data.columns = data.columns.str.strip()

        # YOLO salva più loss (box, cls, dfl).
        # Usiamo 'val/box_loss' e 'train/box_loss' come metriche di confronto.

        epoch = data['epoch']
        val_loss = data['val/dfl_loss']     # Puoi cambiarla con 'val/cls_loss'
        train_loss = data['train/dfl_loss'] # Puoi cambiarla con 'train/cls_loss'

        # Validation Loss (Blu)
        ax.plot(epoch, val_loss, label=f'Validation Loss {modelli[i]}', color=colors_val[i], linewidth=2, linestyle='-')
        # Training Loss (Arancione)
        ax.plot(epoch, train_loss, label=f'Training Loss {modelli[i]}', color=colors_train[i], linewidth=2, linestyle='-')

    except FileNotFoundError:
        print(f"--- ERRORE ---")
        print(f"File non trovato: '{path}'")
        print("-" * 30)
        all_data_found = False
    except KeyError as e:
        print(f"--- ERRORE ---")
        print(f"Errore nel file: {path}. Colonna non trovata: {e}")
        print("-" * 30)
        all_data_found = False

ax.set_xlabel('Epoche')
ax.set_ylabel('Box Loss (Train & Val)')
ax.set_title('Comparison of Training and Validation dfl Loss between YOLO12 Models')
ax.legend(title=' YOLO12 Models (n, m, s)')
ax.grid(True, linestyle='--', alpha=0.6)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if all_data_found:
    plt.tight_layout()
    plt.show()

plt.close()



## Comparison of Precision, Recall and F1 metrics between different models

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics_prf = ['AVG.Precision', 'Avg.Recall', 'Avg.F1']
models = ['Yolo12n', 'Yolo12s', 'Yolo12m'] 
values_prf = np.array([
    [np.mean(resultN.box.p), np.mean(resultS.box.p), np.mean(resultM.box.p)],   # values for Precision
    [np.mean(resultN.box.r), np.mean(resultS.box.r), np.mean(resultM.box.r)],   # values for Recall
    [np.mean(resultN.box.f1), np.mean(resultS.box.f1), np.mean(resultM.box.f1)] # values for F1-score
])
num_metrics_prf = len(metrics_prf)
num_models = len(models)

column_width = 0.2
gap = 0.1
position_bar = np.arange(num_metrics_prf) * (num_models * column_width + gap)

colors = ['#F50C00', '#00FF66', '#007BFF']

fig_prf, ax_prf = plt.subplots(figsize=(10, 6))

for i in range(num_models):
    bars = ax_prf.bar(position_bar + i * column_width, values_prf[:, i], column_width,
                      label=models[i], color=colors[i])

    ax_prf.bar_label(bars, fmt='%.3f', padding=3, fontsize=9)

ax_prf.set_xlabel('metrics')
ax_prf.set_ylabel('Value')
ax_prf.set_title('Comparison of Precision, Recall and F1 metrics between different models')
ax_prf.set_xticks(position_bar + (num_models - 1) * column_width / 2)
ax_prf.set_xticklabels(metrics_prf)
ax_prf.legend(title='Models', loc='lower right')
ax_prf.set_ylim(0, np.max(values_prf) * 1.1)

plt.tight_layout()
plt.show()
plt.close()

## Test for all models

In [ ]:
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML, clear_output
import os
from ultralytics import YOLO

def draw_text_with_background(img, text, org, font, font_scale, text_color, bg_color, thickness, padding=5):
    (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)

    x, y = org
    tl = (x - padding, y - text_h - baseline - padding)
    br = (x + text_w + padding, y + baseline + padding)

    cv2.rectangle(img, tl, br, bg_color, -1)
    cv2.putText(img, text, (x, y), font, font_scale, text_color, thickness)

folder = "/kaggle/input/street-sign-set/StreetSignSet/test/images"

#model = YOLO("/kaggle/input/streetsignsensey12n/pytorch/streetsignsensey12n_350e_final/1/streetsignsense/yolov12n_run/weights/best.pt")
model = YOLO("/kaggle/input/streetsignsensey12s/pytorch/streetsignsensey12s_270e_final/1/streetsignsense/yolo12s_run/weights/best.pt")
#model = YOLO("/kaggle/input/streetsignsensey12m/pytorch/streetsignsensey12m_245e/1/streetsignsense/yolo12m_run/weights/best.pt")

image_files = sorted([f for f in os.listdir(folder) if f.endswith(('.jpg', '.png', '.jpeg'))])

if not image_files:
    print(f"Nessuna immagine trovata nella cartella: {folder}")
else:
    print(f"Trovate {len(image_files)} immagini.")

for filename in image_files:
    image_path = os.path.join(folder, filename)
    print(f"\nRisultati per: {filename} ")

    # ATTENZIONE durante l'inferenza per sfruttare entrambe le GPU T4, si usa l'argomento device='0,1', 
    # in caso di hardware differente occorre modificare o rimuvere l'agormento.
    
    # senza salvataggio
    results = model(image_path, device='0,1')

    # con salvataggio
    #results = model.predict(source=image_path, save=True, device='0,1')

    # con salvataggio e soglie di IoU e confidenza per scartare predizioni incerte,
    #results = model.predict(source=image_path, save=False, iou=0.5, conf=0.5, device='0,1')

    # senza salvataggio e soglie di IoU e confidenza per scartare predizioni incerte,
    #results = model.predict(source=image_path, save=False, iou=0.5, conf=0.5, device='0,1')

    # Immagine Originale
    original_image = cv2.imread(image_path)
    original_image_resized = cv2.resize(original_image, (350,350))

    # Immagine con Predizioni
    prediction_image = results[0].plot() # Questo sostituisce tutto il disegno manuale
    prediction_image_resized = cv2.resize(prediction_image, (350,350))

    # immagine di Ground Truth
    gt_path = os.path.join('//kaggle/usr/lib/sss_groundtruth_annotations/groundTruth/test', filename)
    ground_truth_image = cv2.imread(gt_path)

    if ground_truth_image is None:
        ground_truth_image_resized = cv2.resize(original_image, (350,350))
        draw_text_with_background(ground_truth_image_resized, "GT NOT FOUND", (50, 175), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), (0, 0, 0), 2)
    else:
        ground_truth_image_resized = cv2.resize(ground_truth_image, (350,350))

    title_font = cv2.FONT_HERSHEY_PLAIN

    draw_text_with_background(original_image_resized, "ORIGINAL", (10, 30), title_font, 1, (255, 255, 255), (0,0,0), 2)
    draw_text_with_background(ground_truth_image_resized, "GROUND TRUTH", (10, 30), title_font, 1, (255, 255, 255), (20, 180, 20), 2)
    draw_text_with_background(prediction_image_resized, "PREDICTIONS", (10, 30), title_font, 1, (255, 255, 255), (255, 123, 0), 2)


    concat = cv2.hconcat([original_image_resized, ground_truth_image_resized, prediction_image_resized])
    cv2_imshow(concat)
    print(f"\n_____________________________________________________________________________________________________________________________")
